# Mitandao ya neva ya kurudia

Katika moduli iliyopita, tumekuwa tukitumia uwakilishi wa semantiki tajiri wa maandishi, na kionyeshi rahisi cha mstari juu ya embeddings. Kile usanifu huu unafanya ni kunasa maana iliyojumlishwa ya maneno katika sentensi, lakini hauzingatii **mpangilio** wa maneno, kwa sababu operesheni ya kujumlisha juu ya embeddings huondoa taarifa hii kutoka kwa maandishi ya asili. Kwa kuwa mifano hii haiwezi kuiga mpangilio wa maneno, haiwezi kutatua kazi ngumu zaidi au zenye utata kama vile uzalishaji wa maandishi au kujibu maswali.

Ili kunasa maana ya mlolongo wa maandishi, tunahitaji kutumia usanifu mwingine wa mtandao wa neva, unaoitwa **mtandao wa neva wa kurudia**, au RNN. Katika RNN, tunapitisha sentensi yetu kupitia mtandao moja kwa moja, ishara moja kwa wakati, na mtandao huzalisha hali fulani (**state**), ambayo tunapitisha tena kwenye mtandao pamoja na ishara inayofuata.

Kwa kuzingatia mlolongo wa ishara za pembejeo $X_0,\dots,X_n$, RNN huunda mlolongo wa vizuizi vya mtandao wa neva, na hufundisha mlolongo huu kutoka mwanzo hadi mwisho kwa kutumia kurudisha nyuma. Kila kizuizi cha mtandao huchukua jozi $(X_i,S_i)$ kama pembejeo, na huzalisha $S_{i+1}$ kama matokeo. Hali ya mwisho $S_n$ au matokeo $X_n$ huingia kwenye kionyeshi cha mstari ili kutoa matokeo. Vizuizi vyote vya mtandao vinashiriki uzito sawa, na hufundishwa kutoka mwanzo hadi mwisho kwa kutumia mchakato mmoja wa kurudisha nyuma.

Kwa sababu vekta za hali $S_0,\dots,S_n$ zinapitishwa kupitia mtandao, inaweza kujifunza utegemezi wa mlolongo kati ya maneno. Kwa mfano, wakati neno *si* linatokea mahali fulani katika mlolongo, linaweza kujifunza kukanusha vipengele fulani ndani ya vekta ya hali, na kusababisha kukanusha.

> Kwa kuwa uzito wa vizuizi vyote vya RNN kwenye picha vinashirikiana, picha hiyo hiyo inaweza kuwakilishwa kama kizuizi kimoja (kwenye upande wa kulia) na kitanzi cha maoni ya kurudia, ambacho kinapitisha hali ya matokeo ya mtandao kurudi kwenye pembejeo.

Hebu tuone jinsi mitandao ya neva ya kurudia inaweza kutusaidia kuainisha seti yetu ya data ya habari.


In [1]:
import torch
import torchtext
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)

Loading dataset...
Building vocab...


## Kainishi cha RNN

Katika hali ya RNN rahisi, kila kitengo cha kurudia ni mtandao rahisi wa mstari, ambao huchukua vector ya pembejeo iliyounganishwa na vector ya hali, na kutoa vector mpya ya hali. PyTorch inawakilisha kitengo hiki kwa darasa la `RNNCell`, na mitandao ya seli kama hizo - kama safu ya `RNN`.

Ili kufafanua classifier ya RNN, tutatumia kwanza safu ya embedding kupunguza ukubwa wa msamiati wa pembejeo, kisha kuweka safu ya RNN juu yake:


In [2]:
class RNNClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.rnn = torch.nn.RNN(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,h = self.rnn(x)
        return self.fc(x.mean(dim=1))

> **Note:** Tunatumia safu ya embedding isiyofunzwa hapa kwa urahisi, lakini kwa matokeo bora zaidi tunaweza kutumia safu ya embedding iliyofunzwa awali na Word2Vec au GloVe embeddings, kama ilivyoelezwa katika sehemu ya awali. Ili kuelewa vyema, unaweza kurekebisha msimbo huu ili ufanye kazi na embeddings zilizofunzwa awali.

Katika hali yetu, tutatumia data loader yenye padding, hivyo kila kundi litakuwa na idadi ya mfuatano uliowekwa padding wa urefu sawa. Safu ya RNN itachukua mfuatano wa tensors za embedding, na kutoa matokeo mawili:
* $x$ ni mfuatano wa matokeo ya seli za RNN katika kila hatua
* $h$ ni hali ya mwisho ya siri kwa kipengele cha mwisho cha mfuatano

Kisha tunatumia classifier ya mstari iliyounganishwa kikamilifu ili kupata idadi ya darasa.

> **Note:** RNNs ni ngumu sana kufunza, kwa sababu mara seli za RNN zinapopanuliwa kulingana na urefu wa mfuatano, idadi ya safu zinazohusika katika back propagation huwa kubwa sana. Kwa hivyo tunahitaji kuchagua kiwango kidogo cha kujifunza, na kufunza mtandao kwenye seti kubwa ya data ili kupata matokeo mazuri. Inaweza kuchukua muda mrefu, hivyo kutumia GPU inapendekezwa.


In [3]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)
net = RNNClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.3090625
6400: acc=0.38921875
9600: acc=0.4590625
12800: acc=0.511953125
16000: acc=0.5506875
19200: acc=0.57921875
22400: acc=0.6070089285714285
25600: acc=0.6304296875
28800: acc=0.6484027777777778
32000: acc=0.66509375
35200: acc=0.6790056818181818
38400: acc=0.6929166666666666
41600: acc=0.7035817307692308
44800: acc=0.7137276785714286
48000: acc=0.72225
51200: acc=0.73001953125
54400: acc=0.7372794117647059
57600: acc=0.7436631944444444
60800: acc=0.7503947368421052
64000: acc=0.75634375
67200: acc=0.7615773809523809
70400: acc=0.7662642045454545
73600: acc=0.7708423913043478
76800: acc=0.7751822916666666
80000: acc=0.7790625
83200: acc=0.7825
86400: acc=0.7858564814814815
89600: acc=0.7890513392857142
92800: acc=0.7920474137931034
96000: acc=0.7952708333333334
99200: acc=0.7982258064516129
102400: acc=0.80099609375
105600: acc=0.8037594696969697
108800: acc=0.8060569852941176


## Long Short Term Memory (LSTM)

Moja ya matatizo makubwa ya RNN za kawaida ni tatizo linalojulikana kama **vanishing gradients**. Kwa sababu RNN hufunzwa kutoka mwanzo hadi mwisho kwa kutumia mchakato mmoja wa back-propagation, huwa na ugumu wa kusambaza makosa hadi kwenye tabaka za mwanzo za mtandao, na hivyo mtandao hauwezi kujifunza uhusiano kati ya tokeni za mbali. Njia moja ya kuepuka tatizo hili ni kuanzisha **usimamizi wa hali wazi** kwa kutumia kile kinachoitwa **milango**. Kuna usanifu mawili yanayojulikana zaidi wa aina hii: **Long Short Term Memory** (LSTM) na **Gated Relay Unit** (GRU).

![Picha inayoonyesha mfano wa seli ya long short term memory](../../../../../lessons/5-NLP/16-RNN/images/long-short-term-memory-cell.svg)

Mtandao wa LSTM umeandaliwa kwa namna inayofanana na RNN, lakini kuna hali mbili zinazopitishwa kutoka tabaka moja hadi nyingine: hali halisi $c$, na vekta iliyofichwa $h$. Katika kila kitengo, vekta iliyofichwa $h_i$ inaunganishwa na ingizo $x_i$, na vinaamua nini kitafanyika kwa hali $c$ kupitia **milango**. Kila mlango ni mtandao wa neva wenye uanzishaji wa sigmoid (matokeo katika safu $[0,1]$), ambao unaweza kufikiriwa kama maski ya bitwise inapozidishwa na vekta ya hali. Kuna milango ifuatayo (kutoka kushoto kwenda kulia kwenye picha hapo juu):
* **mlango wa kusahau** huchukua vekta iliyofichwa na kuamua ni vipengele vipi vya vekta $c$ tunavyohitaji kusahau, na vipi kupitisha.
* **mlango wa kuingiza** huchukua baadhi ya taarifa kutoka kwa ingizo na vekta iliyofichwa, na kuziingiza kwenye hali.
* **mlango wa kutoa** hubadilisha hali kupitia safu ya mstari yenye uanzishaji wa $\tanh$, kisha huchagua baadhi ya vipengele vyake kwa kutumia vekta iliyofichwa $h_i$ ili kutoa hali mpya $c_{i+1}$.

Vipengele vya hali $c$ vinaweza kufikiriwa kama bendera fulani zinazoweza kuwashwa na kuzimwa. Kwa mfano, tunapokutana na jina *Alice* katika mlolongo, tunaweza kudhani kuwa linahusu mhusika wa kike, na kuinua bendera katika hali inayoonyesha kuwa tuna nomino ya kike katika sentensi. Tunapokutana baadaye na maneno *and Tom*, tutainua bendera inayoonyesha kuwa tuna nomino ya wingi. Hivyo, kwa kudhibiti hali, tunaweza kudumisha ufuatiliaji wa mali za kisarufi za sehemu za sentensi.

> **Note**: Chanzo kizuri cha kuelewa undani wa LSTM ni makala hii nzuri [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) iliyoandikwa na Christopher Olah.

Ingawa muundo wa ndani wa seli ya LSTM unaweza kuonekana mgumu, PyTorch huficha utekelezaji huu ndani ya darasa `LSTMCell`, na hutoa kitu `LSTM` kuwakilisha safu nzima ya LSTM. Hivyo, utekelezaji wa classifier ya LSTM utakuwa karibu sawa na RNN rahisi ambayo tumeiona hapo juu:


In [4]:
class LSTMClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,(h,c) = self.rnn(x)
        return self.fc(h[-1])

In [5]:
net = LSTMClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.259375
6400: acc=0.25859375
9600: acc=0.26177083333333334
12800: acc=0.2784375
16000: acc=0.313
19200: acc=0.3528645833333333
22400: acc=0.3965625
25600: acc=0.4385546875
28800: acc=0.4752777777777778
32000: acc=0.505375
35200: acc=0.5326704545454546
38400: acc=0.5557552083333334
41600: acc=0.5760817307692307
44800: acc=0.5954910714285714
48000: acc=0.6118333333333333
51200: acc=0.62681640625
54400: acc=0.6404779411764706
57600: acc=0.6520138888888889
60800: acc=0.662828947368421
64000: acc=0.673546875
67200: acc=0.6831547619047619
70400: acc=0.6917897727272727
73600: acc=0.6997146739130434
76800: acc=0.707109375
80000: acc=0.714075
83200: acc=0.7209134615384616
86400: acc=0.727037037037037
89600: acc=0.7326674107142858
92800: acc=0.7379633620689655
96000: acc=0.7433645833333333
99200: acc=0.7479032258064516
102400: acc=0.752119140625
105600: acc=0.7562405303030303
108800: acc=0.76015625
112000: acc=0.7641339285714286
115200: acc=0.7677777777777778
118400: acc=0.77112331081

(0.03487814127604167, 0.7728)

## Sehemu zilizojazwa

Katika mfano wetu, tulilazimika kujaza mfuatano wote kwenye kundi dogo kwa kutumia vekta za sifuri. Ingawa hii husababisha upotevu wa kumbukumbu, kwa RNN ni muhimu zaidi kwamba seli za ziada za RNN zinaundwa kwa ajili ya vipengele vya pembejeo vilivyojazwa, ambavyo vinashiriki katika mafunzo lakini havibebi taarifa muhimu za pembejeo. Ingekuwa bora zaidi kufundisha RNN kwa ukubwa halisi wa mfuatano.

Ili kufanya hivyo, muundo maalum wa uhifadhi wa mfuatano uliojazwa umetambulishwa katika PyTorch. Tuseme tuna kundi dogo la pembejeo lililojazwa ambalo linaonekana kama hili:
```
[[1,2,3,4,5],
 [6,7,8,0,0],
 [9,0,0,0,0]]
```
Hapa 0 inawakilisha thamani zilizojazwa, na vekta halisi ya urefu wa mfuatano wa pembejeo ni `[5,3,1]`.

Ili kufundisha RNN kwa ufanisi na mfuatano uliojazwa, tunataka kuanza mafunzo ya kundi la kwanza la seli za RNN na kundi kubwa (`[1,6,9]`), lakini kisha kumaliza uchakataji wa mfuatano wa tatu, na kuendelea na mafunzo kwa makundi madogo (`[2,7]`, `[3,8]`), na kadhalika. Kwa hivyo, mfuatano uliojazwa unawakilishwa kama vekta moja - katika kesi yetu `[1,6,9,2,7,3,8,4,5]`, na vekta ya urefu (`[5,3,1]`), ambayo tunaweza kuunda upya kundi dogo la awali lililojazwa kwa urahisi.

Ili kuzalisha mfuatano uliojazwa, tunaweza kutumia kazi ya `torch.nn.utils.rnn.pack_padded_sequence`. Tabaka zote za kurudia, ikiwa ni pamoja na RNN, LSTM na GRU, zinaunga mkono mfuatano uliojazwa kama pembejeo, na huzalisha matokeo yaliyofungashwa, ambayo yanaweza kufasiriwa kwa kutumia `torch.nn.utils.rnn.pad_packed_sequence`.

Ili kuweza kuzalisha mfuatano uliojazwa, tunahitaji kupitisha vekta ya urefu kwa mtandao, na kwa hivyo tunahitaji kazi tofauti ya kuandaa makundi madogo:


In [6]:
def pad_length(b):
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch and length sequence itself
    len_seq = list(map(len,v))
    l = max(len_seq)
    return ( # tuple of three tensors - labels, padded features, length sequence
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v]),
        torch.tensor(len_seq)
    )

train_loader_len = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=pad_length, shuffle=True)

Mtandao halisi utakuwa sawa sana na `LSTMClassifier` hapo juu, lakini mchakato wa `forward` utapokea minibatch iliyopangwa pamoja na vector ya urefu wa mfuatano. Baada ya kuhesabu embedding, tunahesabu mfuatano uliopakiwa, kuupitisha kwenye safu ya LSTM, na kisha kufungua tena matokeo.

> **Note**: Kwa kweli hatutumii matokeo yaliyofunguliwa `x`, kwa sababu tunatumia matokeo kutoka kwa tabaka zilizofichwa katika mahesabu yanayofuata. Hivyo basi, tunaweza kuondoa kabisa hatua ya kufungua kutoka kwenye msimbo huu. Sababu ya kuiweka hapa ni ili uweze kurekebisha msimbo huu kwa urahisi, iwapo utahitaji kutumia matokeo ya mtandao katika mahesabu zaidi.


In [7]:
class LSTMPackClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x, lengths):
        batch_size = x.size(0)
        x = self.embedding(x)
        pad_x = torch.nn.utils.rnn.pack_padded_sequence(x,lengths,batch_first=True,enforce_sorted=False)
        pad_x,(h,c) = self.rnn(pad_x)
        x, _ = torch.nn.utils.rnn.pad_packed_sequence(pad_x,batch_first=True)
        return self.fc(h[-1])

In [8]:
net = LSTMPackClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch_emb(net,train_loader_len, lr=0.001,use_pack_sequence=True)


3200: acc=0.285625
6400: acc=0.33359375
9600: acc=0.3876041666666667
12800: acc=0.44078125
16000: acc=0.4825
19200: acc=0.5235416666666667
22400: acc=0.5559821428571429
25600: acc=0.58609375
28800: acc=0.6116666666666667
32000: acc=0.63340625
35200: acc=0.6525284090909091
38400: acc=0.668515625
41600: acc=0.6822596153846154
44800: acc=0.6948214285714286
48000: acc=0.7052708333333333
51200: acc=0.71521484375
54400: acc=0.7239889705882353
57600: acc=0.7315277777777778
60800: acc=0.7388486842105263
64000: acc=0.74571875
67200: acc=0.7518303571428572
70400: acc=0.7576988636363636
73600: acc=0.7628940217391305
76800: acc=0.7681510416666667
80000: acc=0.7728125
83200: acc=0.7772235576923077
86400: acc=0.7815393518518519
89600: acc=0.7857700892857142
92800: acc=0.7895043103448276
96000: acc=0.7930520833333333
99200: acc=0.7959072580645161
102400: acc=0.798994140625
105600: acc=0.802064393939394
108800: acc=0.8051378676470589
112000: acc=0.8077857142857143
115200: acc=0.8104600694444445
118400

(0.029785829671223958, 0.8138166666666666)

> **Kumbuka:** Unaweza kuwa umeona kipengele `use_pack_sequence` tunachopitisha kwa kazi ya mafunzo. Kwa sasa, kazi ya `pack_padded_sequence` inahitaji tensor ya urefu wa mlolongo kuwa kwenye kifaa cha CPU, na hivyo kazi ya mafunzo inahitaji kuepuka kuhamisha data ya urefu wa mlolongo kwenda GPU wakati wa mafunzo. Unaweza kuangalia utekelezaji wa kazi ya `train_emb` katika faili [`torchnlp.py`](../../../../../lessons/5-NLP/16-RNN/torchnlp.py).


## RNN za Mwelekeo Mbili na Tabaka Nyingi

Katika mifano yetu, mitandao yote ya kurudiarudia (recurrent networks) ilifanya kazi kwa mwelekeo mmoja, kutoka mwanzo wa mlolongo hadi mwisho. Inaonekana kuwa ya kawaida, kwa sababu inafanana na jinsi tunavyosoma na kusikiliza hotuba. Hata hivyo, kwa kuwa katika hali nyingi za vitendo tunaweza kufikia mlolongo wa pembejeo kwa nasibu, inaweza kuwa na maana kuendesha hesabu ya kurudiarudia katika pande zote mbili. Mitandao kama hiyo inaitwa **RNN za mwelekeo mbili**, na zinaweza kuundwa kwa kupitisha kipengele `bidirectional=True` kwenye mjengaji wa RNN/LSTM/GRU.

Unaposhughulika na mtandao wa mwelekeo mbili, tunahitaji vekta mbili za hali ya siri, moja kwa kila mwelekeo. PyTorch inahifadhi vekta hizo kama vekta moja yenye ukubwa mara mbili, jambo ambalo ni rahisi sana, kwa sababu kawaida ungetuma hali ya siri inayotokana kwa safu ya mstari iliyounganishwa kikamilifu, na unahitaji tu kuzingatia ongezeko hili la ukubwa wakati wa kuunda safu hiyo.

Mtandao wa kurudiarudia, wa mwelekeo mmoja au wa mwelekeo mbili, unakamata mifumo fulani ndani ya mlolongo, na inaweza kuihifadhi kwenye vekta ya hali au kuipitisha kwenye pato. Kama ilivyo kwa mitandao ya convolutional, tunaweza kujenga safu nyingine ya kurudiarudia juu ya safu ya kwanza ili kukamata mifumo ya kiwango cha juu, iliyojengwa kutoka kwa mifumo ya kiwango cha chini iliyotolewa na safu ya kwanza. Hii inatupeleka kwenye dhana ya **RNN ya tabaka nyingi**, ambayo inajumuisha mitandao miwili au zaidi ya kurudiarudia, ambapo pato la safu ya awali linapitishwa kwa safu inayofuata kama pembejeo.

![Picha inayoonyesha RNN ya tabaka nyingi ya kumbukumbu ya muda mrefu na mfupi](../../../../../translated_images/sw/multi-layer-lstm.dd975e29bb2a59fe.webp)

*Picha kutoka [makala hii nzuri](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3) ya Fernando López*

PyTorch inafanya ujenzi wa mitandao kama hiyo kuwa kazi rahisi, kwa sababu unahitaji tu kupitisha kipengele `num_layers` kwenye mjengaji wa RNN/LSTM/GRU ili kujenga tabaka kadhaa za kurudiarudia moja kwa moja. Hii pia inamaanisha kuwa ukubwa wa vekta ya hali/siri utaongezeka kwa uwiano, na unahitaji kuzingatia hili wakati wa kushughulikia pato la tabaka za kurudiarudia.


## RNNs kwa kazi nyingine

Katika sehemu hii, tumeona kwamba RNNs zinaweza kutumika kwa uainishaji wa mfuatano, lakini kwa kweli, zinaweza kushughulikia kazi nyingi zaidi, kama vile uzalishaji wa maandishi, tafsiri ya mashine, na mengine. Tutazingatia kazi hizo katika sehemu inayofuata.



---

**Kanusho**:  
Hati hii imetafsiriwa kwa kutumia huduma ya tafsiri ya AI [Co-op Translator](https://github.com/Azure/co-op-translator). Ingawa tunajitahidi kuhakikisha usahihi, tafsiri za kiotomatiki zinaweza kuwa na makosa au kutokuwa sahihi. Hati ya asili katika lugha yake ya awali inapaswa kuchukuliwa kama chanzo cha mamlaka. Kwa taarifa muhimu, tafsiri ya kitaalamu ya binadamu inapendekezwa. Hatutawajibika kwa kutoelewana au tafsiri zisizo sahihi zinazotokana na matumizi ya tafsiri hii.
